# Using Jupyter Notebooks with SICStus Prolog

This notebook gives an introduction to the usage of SICStus Prolog in a Jupyter notebook with the
[SICStus Prolog Jupyter Kernel](https://github.com/anbre/sicstus-prolog-jupyter-kernel).<br>

The main requirement for the Prolog Jupyter kernel was for the usage to be as similar to a Prolog REPL as possible.<br>
Therefore, any code to be executed needs to be valid Prolog code and the output mostly resembles the console output.<br>
Additionally, some Jupyter specific messages are printed such as information about the Prolog server (which is responsible for code execution) being halted or restarted.<br>

A code cell can contain more than one Prolog term.<br>
Each of these terms can be a clause definition, directive, or query.<br>
Any variable bindings or other output produced by the terms is displayed in the cell's output.<br>

A main difference compared to the usage in a console is that no user interaction is possible.<br>
Therefore, [computing further solutions](#multiple_solutions) for a goal needs to be handled differently.<br>
Additionally, [debugging](#debugging) cannot be executed as usual.<br>
A major advantage over using a Prolog REPL is that [predicate (re-)definition](#predicate_definition) is more straightforward.<br>
Another convenience the kernel provides is that if the last term of a cell is missing the terminating full-stop, the code can be executed anyway. <br>

This notebook is structured by the features which the Prolog kernel provides.<br>
For each of them, examples are provided which show some possible use cases.<br>
Furthermore, possible peculiarities are explained.

Some of the features correspond to particularities of Prolog that stem from using SLD-resolution as its execution mechanism.<br>
Others are non-standard features that are available in almost every Prolog implementation.<br>
Additionally, there is kernel specific functionality which is provided mostly for user convenience.

Prolog Particularities:
- [Query Execution](#query_execution)
- [Predicate (Re-)Definition](#predicate_definition)
- [Handling Multiple Solutions](#multiple_solutions)

De-Facto Standard Prolog Features:
- [Loading Source Files and Libraries](#load)
- [Running Automated Tests](#automated_tests)
- [Benchmarking Capabilities](#benchmarking)
- [Debugging](#debugging)

Jupyter Convenience Features
- [Introspection](#introspection)
    - [Code completion](#code_completion)
    - [Inspection](#inspection)
    - [Accessing `juypter` documentation](#jupyter_docs)
- [Reset the Prolog State](#reset_state)
- [Accessing Previous Results](#previous_results)
- [Structured Output](#structured_output)

## Prolog Particularities

### Query Execution <a id='query_execution'></a>

If a cell contains a single potential query, it is interpreted as such as opposed to the [definition of a clause](#predicate_definition).<br>
Additionally, terms starting with `?-` and directives starting with `:-` are evaluated as queries even if they appear in a cell with multiple other terms.

If a query succeeds and binds any variables, the bindings are shown in the output of the cell.<br>
Analogously, if there are no bindings, `yes` is printed and if a query fails, `no` is output.

In [1]:
X = [1,2,3], append(X, [4,5,6], Z).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">X = [1,2,3],
Z = [1,2,3,4,5,6]</pre>

In [ ]:
?- member(2, [1, 2, 3]).
?- member(4, [1, 2, 3])

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">yes</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
color:red; font-weight:bold;">no</pre>

If a query causes an exception, the exception message is printed in red.

In [ ]:
7 is 1 + y.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
color:red; font-weight:bold;">! Type error in argument 2 of (is)/2
! expected evaluable, but found y/0
! goal:  7 is 1+y
</pre>

If the execution produces any output, it is displayed preceding the other information.

In [4]:
format('Using ~w Prolog!~n', ['SICStus']).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">Using SICStus Prolog!</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">yes</pre>

Just like in a Prolog REPL, no variable bindings are output for directives.

In [5]:
:- member(M, [1, 2, 3]), print(M).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">1</pre>

### Predicate (Re-)Definition <a id='predicate_definition'></a>

If a cell contains more than one term, in general, all terms are added as dynamic facts to the data base.<br>
The only exceptions are terms starting with `?-` or `:-` and PL-Unit test definitions.<br>

All defined clauses can be module name expanded such as `Module:Head :- Body.`. <br>
If no module name is defined, the module `user` is chosen by default.<br>
In order to let the user know the exact predicate which was defined, a message containing the module name and predicate name and arity is output.

In [6]:
app([], Res, Res).
app([H|T], List, [H|Res]) :-
  app(T, List, Res).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Asserting clauses for user:app/3
</pre>

If users want to assert a single fact, they can achieve it by writing a term of the form `Head :- Body.`.<br>
Also note that it is possible to define and query a predicate in the same cell.

In [7]:
app(L1, L2, L3, Res) :-
  app(L2, L3, R1),
  app(L1, R1, Res).
  
?- app([1,2], [3,4], [5,6], R).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Asserting clauses for user:app/4
</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">R = [1,2,3,4,5,6]</pre>

The use of Prolog in a Jupyter application is meant for interactive programming.<br>
This involves writing, testing and re-writing clauses rather than only adding new clauses to the fact database.<br>
Therefore, when clauses are defined for a dynamic predicate for which there are existing ones, these are retracted first.<br>
Thus, all clauses of a predicate need to be defined in one cell.<br>

In case clauses were retracted, the user is informed.<br>
This is done by a message preceding all other output.<br>
The message can be expanded in order to see the exact clauses which were retracted.<br>

In [8]:
user:app([], Res, Res) :- !.
user:app([Head|Tail], List, [Head|Res]) :-
  app(Tail, List, Res).

Some previously defined clauses were retracted:
user:app/3:
app([], A, A).
app([A|B], C, [A|D]) :-
        app(B, C, D).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Asserting clauses for user:app/3
</pre>

Additionally, clauses can be defined in the form of DCG rules.

In [9]:
number(Number) --> "+", number(Number).
number(NegativeNumber) --> "-", number(Number), {NegativeNumber is -Number}.
number(Number) --> [Digit], {"0"=<Digit, Digit=<"9", Number is Digit - "0"}.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Asserting clauses for user:number/3
</pre>

In [10]:
atom_codes('-1', _Codes), phrase(number(N), _Codes).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">N = -1</pre>

### Handling Multiple Solutions <a id='multiple_solutions'></a>

When a query succeeds with a choice point, one can request further solutions via backtracking.<br>
The Jupyter kernel mimics this by providing the predicate `jupyter:retry/0`.<br>
Whenever a query is executed, it is seen as the active query as long as there might be further solutions for it.<br>
If there is an active query, a `retry` may trigger backtracking to compute the next solution.<br>
In order to let the user know which query is retried, a corresponding message is output.<br>

Additionally, the predicate `jupyter:cut/0` is exposed.<br>
It cuts off choicepoints of the active execution and marks an older query which might have open choice points as active.<br>
The user is informed about the new active query.

Since backtracking is a Prolog feature which is used frequently, these two predicates can be called without stating the module and cannot be redefined in the `user` module.<br>

It needs to be noted that both predicates do not work unless they are the only goal in a query.<br>

In [11]:
?- member(M, [1, 2]).
?- member(M, [a, b, c]).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">M = 1</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">M = a</pre>

In [12]:
retry.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Retrying goal: member(M,[a,b,c])
</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">M = b</pre>

In [13]:
cut.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Successfully cut
% The new active goal is: member(M,[1,2])</pre>

In [14]:
jupyter:retry.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Retrying goal: member(M,[1,2])
</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">M = 2</pre>

The `retry` and `cut` mechanism can be useful when:
- Computing the solution of a query
- Further inspecting the solution with a different query
- Computing the next solution and further inspecting it

This is especially useful when the computation of the first solution took so long that it should not be executed again.

For an explanation of `$Var` terms, see [Accessing Previous Results](#previous_results).


In [15]:
is_odd(X) :-
  1 is X mod 2.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Asserting clauses for user:is_odd/1
</pre>

In [16]:
member(M, [1, 2, 3, 4, 5]).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">M = 1</pre>

In [17]:
is_odd($M).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">M = 1</pre>

In [ ]:
?- cut.
?- retry.
?- is_odd($M).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Successfully cut
% The new active goal is: member(M,[1,2,3,4,5])</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Retrying goal: member(M,[1,2,3,4,5])
</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">M = 2</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
color:red; font-weight:bold;">no</pre>

Note that this time no `cut` is needed because the previous query failed.

In [19]:
?- retry.
?- is_odd($M).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Retrying goal: member(M,[1,2,3,4,5])
</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">M = 3</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">M = 3</pre>

## De-Facto Standard Prolog Features

### Loading Source Files and Libraries <a id='load'></a>

Loading source files and libraries works in the same way as on a Prolog REPL.<br>
Predicates are always re-defined when encountered (as user interaction is not possible during the load).

In [20]:
use_module(library(lists)).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% module lists imported into user</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">yes</pre>

In [21]:
append([[1], [2], [3]], L). 

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">L = [1,2,3]</pre>

Note that loading a library and using operators from that library does not work in a single query.<br>
The term needs to be read by the built-in predicate `read_term/3`, which throws a syntax error when encountering undefined operators.

In [ ]:
use_module(library(clpfd)), X #< 10, X #> 5.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
color:red; font-weight:bold;">! Syntax error in read_term/3
! operator expected after expression
! in line 1
! use_module ( library ( clpfd ) ) , X 
! <<here>>
! #< 10 , X #> 5 . 
</pre>

In [23]:
use_module(library(clpfd)).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% loading /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/clpfd.po...
% module clpfd imported into user
%  loading /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/atts.po...
%  module attributes imported into clpfd
%   module types imported into attributes
%  loaded /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/atts.po in module attributes, 3 msec 30544 bytes
%  compiling /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/avl.pl...
%   module avl imported into clpfd
%  compiled /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/avl.pl in module avl, 16 msec 62768 bytes
%  module lists imported into clpfd
%  loading /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/ordsets.po...
%  module ordsets imported into clpfd
%  loaded /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/ordsets.po in module ordsets, 1 msec 45904 bytes
%  loading /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/trees.po...
%  module trees imported into clpfd
%  loaded /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/trees.po in module trees, 2 msec 11488 bytes
%  module types imported into clpfd
%  loading /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/terms.po...
%  module terms imported into clpfd
%   module types imported into terms
%   module avl imported into terms
%  loaded /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/terms.po in module terms, 0 msec 46304 bytes
%  loading /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/timeout.po...
%  module timeout imported into clpfd
%  loaded /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/timeout.po in module timeout, 0 msec 976 bytes
%  loading foreign resource /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/x86_64-linux-glibc2.17/clpfd.so in module clpfd
% loaded /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/clpfd.po in module clpfd, 89 msec 2137296 bytes</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">yes</pre>

In [24]:
X #< 10, X #> 5.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">X in 6..9</pre>

### Running Automated Tests <a id='automated_tests'></a>

`library(plunit)` can be used to define and run automated tests.<br>
Tests can either be defined in a file which is loaded or in a cell.<br>
In the latter case, any definition of a `test/1` or `test/2` clause needs to be preceded by a `begin_tests` directive.<br>
Additionally, if there is an optional `end_tests` directive, it needs to follow the test clauses.<br>
Otherwise, they are not recognized as such.<br>

In [25]:
:- begin_tests(list). 

test(list) :-
  lists:is_list([]).

:- end_tests(list).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% compiling /media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/jupyter_tests.pl...
%  loading /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/plunit.po...
%  module plunit imported into user
%   module lists imported into plunit
%   module types imported into plunit
%  loaded /usr/local/sicstus4.5.1/bin/sp-4.5.1/sicstus-4.5.1/library/plunit.po in module plunit, 5 msec 129264 bytes
% compiled /media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/jupyter_tests.pl in module user, 11 msec 157440 bytes
% Loaded the test file</pre>

In [26]:
run_tests.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% PL-Unit: list 
% /media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/jupyter_tests.pl:2:
% 	test list: succeeded (det) in 0.00 seconds
% done
% 1 tests passed</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">yes</pre>

It needs to be noted that after defining new test clauses in a different cell, the test unit of the previous ones still exists, but not the tests themselves.<br>
Therefore, all tests which are to be run at the same time need to be defined in one cell.<br>

In [27]:
:- begin_tests(number). 

test(number_failing, [fail]) :-
  number([]).
  
:- run_tests.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% compiling /media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/jupyter_tests.pl...
% compiled /media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/jupyter_tests.pl in module user, 4 msec 2928 bytes
% Loaded the test file</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% PL-Unit: list 
% done
% PL-Unit: number 
% /media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/jupyter_tests.pl:2:
% 	test number_failing: succeeded (det) in 0.00 seconds
% done
% 1 tests passed</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">yes</pre>

As mentioned above, redefinitions of predicates are always performed.<br>
A file containing `plunit` test clauses defines the predicates `'unit body'/4` and `'unit test'/5`.<br>
If test files with different names are loaded, these predicates are redefined and corresponding messages are output.<br>
When defining tests in a cell, they are written to a file named jupyter_tests.pl which is loaded.<br>
Since this file is named the same for every cell containing tests, when running multiple cells with test definitions, the redefinition warnings are not output for every execution.<br>

In [28]:
[test].

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% compiling /media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/test.pl...
* The procedure 'unit body'/4 is being redefined.
*     Old file: /media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/jupyter_tests.pl
*     New file: /media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/test.pl
* Approximate lines: 12-13, file: '/media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/test.pl'
% compiled /media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/test.pl in module user, 5 msec 8464 bytes</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">yes</pre>

In [29]:
run_tests.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% PL-Unit: list 
% done
% PL-Unit: number 
% done
% PL-Unit: test 
% /media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/test.pl:13:
% 	test a: succeeded (det) in 0.00 seconds
% /media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/test.pl:16:
% 	test b: succeeded (det) in 0.00 seconds
% /media/storage/Uni/Masterarbeit/sicstus_kernel/notebooks/test.pl:19:
% 	test c: succeeded (det) in 0.00 seconds
% done
% 3 tests passed</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">yes</pre>

### Benchmarking Capabilities <a id='benchmarking'></a>

Whenever a query is executed, its runtime is stored in a database.<br>
It can be accessed by calling `jupyter:previous_query_time(Goal, Time)`.<br>
`Goal` is the previous goal which was executed and `Time` is the time in milliseconds it took the query to complete.

In [30]:
member(M, [1,2,3]).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">M = 1</pre>

In [31]:
jupyter:previous_query_time(Goal, Time).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">Goal = member(M,[1,2,3]),
Time = 0</pre>

### Debugging <a id='debugging'></a>

In a Jupyter notebook, debugging cannot be performed interactively as user input is not supported during the execution of a cell.<br>
Thus, switching on trace mode with `trace/0` would cause the server to stop at an invocation (and a restart is required).<br>
However, the call stack can be accessed by defining breakpoints which cause debugging messages to be printed.<br>
As this mechanism might be difficult, the predicate `jupyter:trace(Goal)` was implemented.<br>
It switches on trace mode, calls the goal `Goal` and switches debug mode off.<br>
By default, all ports are unleashed and included in the output, which means that no user interaction is requested when a breakpoint is activated.<br>
In order to be recognized as a special goal, the call of `jupyter:trace/1` needs to be the only one in a term.<br>

In [32]:
app([], Res, Res) :- !.
app([Head|Tail], List, [Head|Res]) :-
  app(Tail, List, Res).
  
app(L1, L2, L3, Res) :-
  app(L2, L3, R1),
  app(L1, R1, Res).

Some previously defined clauses were retracted:
user:app/3:
app([], A, A) :- !.
app([A|B], C, [A|D]) :-
        app(B, C, D).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Asserting clauses for user:app/3
</pre>

Some previously defined clauses were retracted:
user:app/4:
app(A, B, C, D) :-
        app(B, C, E),
        app(A, E, D).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Asserting clauses for user:app/4
</pre>

In [33]:
jupyter:trace(app([1], [2], R)).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">        1      1 Call: app([1],[2],_515343)
        2      2 Call: app([],[2],_519127)
        2      2 Exit: app([],[2],[2])
        1      1 Exit: app([1],[2],[1,2])</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">R = [1,2]</pre>

As mentioned above, when a breakpoint is activated, user interaction must not be required.<br>
In order to prevent this, a breakpoint should be created with the option `proceed` so that on activation the execution is continued without stopping.<br>

In [34]:
add_breakpoint(pred(app/3)-[print-[1], proceed], _). % print the first argument in the debugging message

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% The debugger will first zip -- showing spypoints (zip)
% Conditional spypoint for user:app/3 added, BID=1</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">yes</pre>

In [35]:
app([1], [2], R).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
"> *      4      1 Call: ^1 [1]
 *      5      2 Call: ^1 []
 *      5      2 Exit: ^1 []
 *      4      1 Exit: ^1 [1]</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">R = [1,2]</pre>

All created breakpoints are still active during a `jupyter:trace/1` call.

In [36]:
jupyter:trace((app([1], [2], [3], [1,2,3]))).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">        6      1 Call: app([1],[2],[3],[1,2,3])
 *      7      2 Call: ^1 [2]
 *      8      3 Call: ^1 []
 *      8      3 Exit: ^1 []
 *      7      2 Exit: ^1 [2]
 *      9      2 Call: ^1 [1]
 *     10      3 Call: ^1 []
 *     10      3 Exit: ^1 []
 *      9      2 Exit: ^1 [1]
        6      1 Exit: app([1],[2],[3],[1,2,3])</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">yes</pre>

## Jupyter Convenience Features

### Introspection <a id='introspection'></a>

#### Code Completion (Tab) <a id='code_completion'></a>
In JupyterLab as well as Jupyter Notebook, code completion for the token at the current cursor position can be requested by pressing the Tab key.<br>
If there is a single possible match, the code preceding the cursor is replaced directly.<br>
Otherwise, a list of options is shown from which the user can choose one.<br>
Completion can be used for predicates which are built-in or exported by a loaded module.<br>
It is also possible to display all predicates of a module, which is especially useful for retrieving all the special predicates defined by the module `jupyter`.

#### Inspection (Shift+Tab) <a id='inspection'></a>
By pressing Shift+Tab, inspection for the token preceding the current cursor position can be requested.<br>
In contrast to some other kernels, the SICStus kernel does not show the docstring for most predicates right away.<br>
Instead, for all predicates which the [Predicate Index page](https://sicstus.sics.se/sicstus/docs/latest/html/sicstus.html/Predicate-Index.html) lists,
a link to the documentation of the corresponding predicate is shown if the predicate's name contains the current token.<br>
The data shown about the predicate is the same as shown on the web page.<br>
In most cases, it includes the predicate's name, arity, and information such as if it is built-in or the library's name.<br>
For JupyterLab, clickable hyperlinks can be displayed;
since this is not possible for Jupyter Notebook, the links are only given to be copied.<br>
However, for predicates from the module `jupyter`, the documentation is displayed.

#### Accessing `jupyter` documentation <a id='jupyter_docs'></a>

Most of the features below are provided by the module `jupyter` which is used by the Prolog server.<br>
In addition to the inspection, the documentation of all predicates defined in this module can be accessed with `jupyter:help/0`.<br>
If the predicate is called without the module name expansion and it is not defined in the current module `user`, the corresponding error message is displayed followed by a note that there is `jupyter:help/0`.<br>
However, if `user:help/0` is defined, calling it works as expected without any special output for the `jupyter` predicate.<br>

In [ ]:
help.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
color:red; font-weight:bold;">! Existence error in user:help/0
! procedure user:help/0 does not exist
! goal:  user:help
! 
! However, there is the predicate jupyter:help/0
</pre>

In [38]:
jupyter:help

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">jupyter:help

    Outputs the documentation for all predicates from module jupyter.

--------------------------------------------------------------------------------

jupyter:halt or halt

    Shuts down the running Prolog process.

    The next time code is to be executed, a new process is started.
    Everything defined in the database before does not exist anymore.

    Corresponds to the functionality of halt/0.
    Has the same effect as interrupting or restarting the Jupyter kernel.

--------------------------------------------------------------------------------

jupyter:retry or retry

    Causes backtracking of the latest active query.

    In general, the previous query is the active one.
    However, the previous active query can be activated again.
    This can be done by cutting off choicepoints with jupyter:cut/0.
    This is also the case if a retry/0 encounters no further solutions.

    Needs to be the only goal of a query.

--------------------------------------------------------------------------------

jupyter:cut or cut

    Cuts off the choicepoints of the latest active query.

    In general, the previous query is the active one.
    However, the previous active query can be activated again.
    This can be done by cutting off choicepoints with jupyter:cut/0.
    This is also the case if a retry/0 encounters no further solutions.

    A further retry/0 call causes backtracking of the previous active goal.

    Needs to be the only goal of a query.

--------------------------------------------------------------------------------

jupyter:trace(+Goal)

    Prints the trace of the goal Goal.
    By default, no port is leashed so that no user interaction is requested.
    All previously set breakpoints are still active.

    Needs to be the only goal of a query in order to work as expected.

--------------------------------------------------------------------------------

jupyter:print_variable_bindings

    Prints variable bindings from previous queries.
    For each variable, the latest value it was bound to is shown.

    The variable value can be accessed with a $Var term by any query.
    In that case, the term is replaced by the value.
    If there is no previous value, an error message is printed.

--------------------------------------------------------------------------------

jupyter:print_table(+Goal)

    Computes all results of the goal Goal with findall/3.
    These are printed in a table.

    Needs to be the only goal of a query.

    Example: jupyter:print_table(prolog_flag(FlagName, Value)).

--------------------------------------------------------------------------------

jupyter:print_table(+ValuesLists, +VariableNames)

    Prints a table of the values in ValuesLists.

    ValuesLists is a list of lists of the same length.
    Each list corresponds to one line of the table.

    VariableNames is used to fill the header of the table.
    If VariableNames=[], capital letters are used.
    Otherwise, VariableNames needs to be a list of ground terms.
    It needs to be of the same length as the values lists.

    Needs to be the only goal of a query.

    Can be used with a predicate like findall/3, but not directly.
    Instead, a previous binding can be accessed with a $Var term.

    Examples:
        jupyter:print_table([[10,100],[20,400],[30,900]], ['X', 'Y']).
        jupyter:print_table($ResultLists, []).

--------------------------------------------------------------------------------

jupyter:previous_query_time(-Goal, -Runtime)

    Goal is the previously executed goal.
    Time is the time in milliseconds it took the query to complete.

--------------------------------------------------------------------------------

jupyter:print_previous_queries(+Ids)

    Prints previous queries which were exectued in requests with IDs in Ids.

    Any $Var terms might be replaced by the variable's name.
    This is the case if a previous query with ID in Ids contains Var.
    Otherwise, $Var is not replaced.</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">yes</pre>

### Reset the Prolog State <a id='reset_state'></a>

All code is executed by a Prolog server.<br>
As long as the server is running, the database state with all potential side effects such as clause assertions is in effect.<br>
However, the user might want to undo all those and reset everything to an initial state.<br>
There are several ways to do this.<br>

The most natural and Prolog-like way is to call the built-in predicate `halt` or the Jupyter version `jupyter:halt`.<br>
It should be noted that this predicate needs to be the only one of a query so that it can be determined as a special query and handled correctly.<br>
If it does work as expected, a success message is output.<br>

Additionally, in JupyterLab as well as Jupyter Notebook, there are buttons for interrupting the kernel and restarting it.<br>
The same can also be achieved by pressing *I+I* and *0+0* respectively.<br>
All of these options cause the Prolog server to be stopped.<br>
The next time code is executed, a message about the restart of the server is output to let the user know that everything defined so far has been undone.<br>

Restarting the server is especially useful when all cells of a notebook are to be run and not all of them are to work on the same Prolog database.

In [39]:
halt.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Successfully halted</pre>

After the server restart, the previously defined predicate `app/4` does not exist anymore.

In [ ]:
app([1,2], [3,4], [5,6], R).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
color:red">% The Prolog server was restarted</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
color:red; font-weight:bold;">! Existence error in user:app/4
! procedure user:app/4 does not exist
! goal:  user:app([1,2],[3,4],[5,6],_14305)
</pre>

### Accessing Previous Results <a id='previous_results'></a>

SWI-Prolog provides functionality of reusing top-level bindings.<br>
Roughly, when a top-level goal succeeds, its bindings are asserted in a database.<br>
By using `$Var` in a top-level query, the latest binding for the variable `Var` can be accessed.<br>
This functionality is also provided by the SICStus Prolog Jupyter kernel.<br>
Additionally, there is the predicate `jupyter:print_variable_bindings/0` that outputs all stored variable bindings.<br>

In [41]:
halt.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">% Successfully halted</pre>

In [42]:
X = 1.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
color:red">% The Prolog server was restarted</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">X = 1</pre>

In [43]:
Y is $X + 2.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">Y = 3,
X = 1</pre>

In [44]:
jupyter:print_variable_bindings.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">$X =        1
$Y =        3</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">yes</pre>

Further, all executed queries are stored internally.<br>
When writing a new predicate, a user might test its subgoals gradually in different cells,
potentially using `$Var` terms to access previous values.<br>
Once all the parts are written, the predicate `jupyter:print_previous_queries(Ids)` can be called to access the previous queries with IDs in the list Ids.<br>
They are printed in a way that they can easily be copied to a cell and executed right away or expanded with a head to define a predicate.<br>
If a query contains a `$Var` term and one of the previously printed queries contains the variable `Var`, the term is replaced by the variable name.<br>

In [45]:
member(Member, [10, 20, 30]).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">Member = 10</pre>

In [46]:
Square is $Member * $Member.

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">Square = 100,
Member = 10</pre>

In [47]:
jupyter:print_previous_queries([45, 46]).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
">  member(Member,[10,20,30]),
  Square is Member*Member.</pre>

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">yes</pre>

### Structured Output <a id='structured_output'></a>

There are two special predicates to output tables: `jupyter:print_table/1` and `jupyter:print_table/2`.

The predicate `jupyter:print_table(Goal)` can be used to compute all results of the goal `Goal` with `findall/3` and display them in a table.<br>
The table contains a column for each variable occurring in it and a line for each result.<br>

In [48]:
jupyter:print_table((member(Member, [10,20]), Square is Member*Member)).

Member | Square | 
:- | :- | 
10 | 100 | 
20 | 400 | 

In order to fill a table with other data than results of `findall/3`, there is the predicate `jupyter:print_table(ValuesLists, VariableNames)`.<br>
`ValuesLists` is expected to be a list of lists where each of this lists corresponds to one line of the table.<br>
Therefore, all the lists need to be of the same length.<br>
Furthermore, `VariableNames` is used to provide the column headers and needs to be a list of ground terms of the same length as well unless it equals `[]`.<br>
In the latter case, the headers contain capital letters starting from `A`.<br>

In [49]:
jupyter:print_table([[10,100],[20,400],[30,900]], []).

A | B | 
:- | :- | 
10 | 100 | 
20 | 400 | 
30 | 900 | 

It needs to be noted that both of the predicates do not work properly unless they are the only goal in a query.<br>
Therefore, the value for `ValuesLists` cannot be computed in the same query.<br>
However, the mechanism of reusing bindings from previous queries can be used.<br>

In [50]:
findall([M, S], (member(M, [10,20]), S is M*M), ResultLists).

<pre style="
font-family: Menlo, Consolas, 'DejaVu Sans Mono', monospace;
font-size: 13px;
line-height: 1.3077;
font-weight: bold;">ResultLists = [[10,100],[20,400]]</pre>

In [51]:
jupyter:print_table($ResultLists, ['Member', 'Square']).

Member | Square | 
:- | :- | 
10 | 100 | 
20 | 400 | 